## 필요 모듈 설치

In [ ]:
!pip uninstall -y segmentation-models-pytorch
!pip install pretrainedmodels==0.7.4
!pip install efficientnet_pytorch==0.6.3
!pip install timm==0.4.12
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir

!pip install git+https://github.com/qubvel/segmentation_models.pytorch

Found existing installation: segmentation-models-pytorch 0.2.1
Uninstalling segmentation-models-pytorch-0.2.1:
  Successfully uninstalled segmentation-models-pytorch-0.2.1
  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-cmg0w7ii
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-cmg0w7ii
  Cloning https://github.com/qubvel/segmentation_models.pytorch to /tmp/pip-req-build-kgl3357t
  Running command git clone -q https://github.com/qubvel/segmentation_models.pytorch /tmp/pip-req-build-kgl3357t
  Created wheel for segmentation-models-pytorch: filename=segmentation_models_pytorch-0.2.1-py3-none-any.whl size=88649 sha256=6d3df987b8793bfe2f3be44529501c5cefc1357e0035b67112b8e46d1d31fd8e
  Stored in directory: /tmp/pip-ephem-wheel-cache-y07_65ve/wheels/fa/c5/a8/1e8af6cb04a0974db8a4a156ebd2fdd1d99ad2558d3fce49d4
Successfully built segmentation-models-pytorch


## 구글 드라이브 연동

In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset
import albumentations as albu
import torch
import numpy as np
import segmentation_models_pytorch as smp

import json

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
workspace_path = '/gdrive/My Drive/Colab Notebooks/SIA'
segmentation_path = os.path.join(workspace_path, 'segmentation_models')

In [ ]:
import sys
sys.path.append(segmentation_path)  # segmentation 소스코드 경로 설정

In [ ]:
#기존 파일 위치 
train_path = os.path.join(workspace_path, 'LV2_training_set')
x_train_dir = os.path.join(train_path, 'images')
y_train_building_dir = os.path.join(train_path, 'labels/building')
y_train_road_dir = os.path.join(train_path, 'labels/road')

train_path = os.path.join(workspace_path, 'LV2_validation_set')
x_valid_dir = os.path.join(train_path, 'images')
y_valid_building_dir = os.path.join(train_path, 'labels/building')
y_valid_road_dir = os.path.join(train_path, 'labels/road')

In [ ]:
#json으로 변환한 파일 위치
train_path = os.path.join(workspace_path, 'training_set')
x_train = os.path.join(train_path, 'images')
y_train = os.path.join(train_path, 'labels')

train_path = os.path.join(workspace_path, 'validation_set')
x_valid = os.path.join(train_path, 'images')
y_valid = os.path.join(train_path, 'labels')


In [ ]:
# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()

##Create Mask and Image of Building+Road

In [ ]:
def get_polygon(p):
  test_list1 = []
  for i in range(0,len(p),2):
    test_list2 = [eval(p[i]),eval(p[i+1])]
    test_list2 = list(map(int, test_list2 ))
    test_list1.append(test_list2)
  polygon = np.array(test_list1)
  return polygon

In [ ]:
def createLabel(path1, path2, path3):
  file_list_building = sorted(os.listdir(path1))
  file_list_road = sorted(os.listdir(path2))

  file_list = list(set(file_list_building).intersection(file_list_road))
  print(len(file_list))

  dict_list_building = []
  for filename in file_list:
        if filename.endswith('.json'):
                with open(os.path.join(path1, filename), 'r') as f:
                      dict_list_building.append(json.load(f))

  dict_list_road = []
  for filename in file_list:
        if filename.endswith('.json'):
                with open(os.path.join(path2, filename), 'r') as f:
                      dict_list_road.append(json.load(f))

  for json_number in range(0,len(file_list)):
    json_data_building = dict_list_building[json_number]
    json_data_road = dict_list_road[json_number]

    mask1 = np.zeros((1024,1024,3),dtype = np.uint8)

  #road part
    feature_size_road = len(json_data_road["features"])
    for i in range(0, feature_size_road):
      test = json_data_road["features"][i]["properties"]["road_imcoords"].split(',')
      if test!=['']:
          polygon = get_polygon(test)
          cv2.fillPoly(mask1,[polygon], color=(255,255,255))
  #building part
    feature_size_building = len(json_data_building["features"])
    for i in range(0, feature_size_building):
      test = json_data_building["features"][i]["properties"]["building_imcoords"].split(',')
      if test!=['']:
          polygon = get_polygon(test)
          cv2.fillPoly(mask1,[polygon], color=(200,200,200))
          
    file_name = json_data_building["features"][0]["properties"]["image_id"]
    print(json_number, file_name)
    
    plt.imsave(os.path.join(path3, file_name), mask1)

In [ ]:
createLabel(y_valid_building_dir, y_valid_road_dir, y_valid)
createLabel(y_train_building_dir, y_train_road_dir, y_train)

212
0 BLD00885_PS3_K3A_NIA0277.png
1 BLD02064_PS3_K3A_NIA0374.png
2 BLD01449_PS3_K3A_NIA0373.png
3 BLD02229_PS3_K3A_NIA0374.png
4 BLD02063_PS3_K3A_NIA0374.png
5 BLD00723_PS3_K3A_NIA0277.png
6 BLD01189_PS3_K3A_NIA0373.png
7 BLD01000_PS3_K3A_NIA0277.png
8 BLD00135_PS3_K3A_NIA0276.png
9 BLD02924_PS3_K3A_NIA0375.png
10 BLD01863_PS3_K3A_NIA0374.png
11 BLD01701_PS3_K3A_NIA0373.png
12 BLD02650_PS3_K3A_NIA0375.png
13 BLD02273_PS3_K3A_NIA0374.png
14 BLD02543_PS3_K3A_NIA0375.png
15 BLD02176_PS3_K3A_NIA0374.png
16 BLD00740_PS3_K3A_NIA0277.png
17 BLD02927_PS3_K3A_NIA0375.png
18 BLD00491_PS3_K3A_NIA0276.png
19 BLD00972_PS3_K3A_NIA0277.png
20 BLD01088_PS3_K3A_NIA0277.png
21 BLD02941_PS3_K3A_NIA0375.png
22 BLD01030_PS3_K3A_NIA0277.png
23 BLD01338_PS3_K3A_NIA0373.png
24 BLD02060_PS3_K3A_NIA0374.png
25 BLD03363_PS3_K3A_NIA0376.png
26 BLD02819_PS3_K3A_NIA0375.png
27 BLD03029_PS3_K3A_NIA0375.png
28 BLD03467_PS3_K3A_NIA0376.png
29 BLD00789_PS3_K3A_NIA0277.png
30 BLD02651_PS3_K3A_NIA0375.png
31 BLD02328_PS

## Create Mask and Image of Building, Road(사용x)

In [ ]:
# def get_polygon(p):
#   test_list1 = []
#   for i in range(0,len(p),2):
#     test_list2 = [eval(p[i]),eval(p[i+1])]
#     test_list2 = list(map(int, test_list2 ))
#     test_list1.append(test_list2)
#   polygon = np.array(test_list1)
#   return polygon

# def building_Color(test_id):
#   if test_id == '1' :
#     return (0,0,255)
#   elif test_id == '2' :
#     return (0,255,0)
#   elif test_id == '3' :
#     return (255,0,0)
#   elif test_id == '4' :
#     return (255,0,255)
#   elif test_id == '5' :
#     return (0,255,255)
#   elif test_id == '6' : 
#     return (255,255,0)

# def road_Color(test_id):
#   if test_id == '1' :
#     return (97,97,97)
#   elif test_id == '2' :
#     return (97,97,0)
#   elif test_id == '3' :
#     return (0,97,97)
#   elif test_id == '4' :
#     return (97,0,97)
#   elif test_id == '5' :
#     return (97,0,0)
#   elif test_id == '6' : 
#     return (0,97,0)

In [ ]:
# def createFile(path1, path2, path3, path4, path5, path6):
#   file_list_building = sorted(os.listdir(path1))
#   file_list_road = sorted(os.listdir(path2))


#   print("파일리스트 생성 중...")
#   dict_list_building = []
#   for filename in file_list_building:
#         if filename.endswith('.json'):
#                 with open(os.path.join(path1, filename), 'r') as f:
#                       dict_list_building.append(json.load(f))

#   dict_list_road = []
#   for filename in file_list_road:
#         if filename.endswith('.json'):
#                 with open(os.path.join(path2, filename), 'r') as f:
#                       dict_list_road.append(json.load(f))
#   print(len(dict_list_building))
#   print(len(dict_list_road))

#   print("라벨 생성 중...")
#   for json_number in range(0,len(dict_list_building)):
#         json_data = dict_list_building[json_number]
#         mask1 = np.zeros((1024,1024,3),dtype = np.uint8)
#         feature_size_building = len(json_data["features"])
#         for i in range(0, feature_size_building):
#           test = json_data["features"][i]["properties"]["building_imcoords"].split(',')
#           if test!=['']:
#               polygon = get_polygon(test)
#               cv2.fillPoly(mask1,[polygon], color=building_Color(json_data["features"][i]["properties"]["type_id"]))
#               cv2.polylines(mask1,[polygon], True, (255,255,255),3)
#         file_name = json_data["features"][0]["properties"]["image_id"]
#         print(json_number, file_name)
#         plt.imsave(os.path.join(path3, file_name), mask1)

#   for json_number in range(0,len(dict_list_road)):
#         json_data = dict_list_road[json_number]
#         mask1 = np.ones((1024,1024,3),dtype = np.uint8)
#         feature_size_road = len(json_data["features"])
#         for i in range(0, feature_size_road):
#           test = json_data["features"][i]["properties"]["road_imcoords"].split(',')
#           if test!=['']:
#               polygon = get_polygon(test)
#               cv2.fillPoly(mask1,[polygon], color=road_Color(json_data["features"][i]["properties"]["type_id"]))
#         file_name = json_data["features"][0]["properties"]["image_id"]
#         print(json_number, file_name)
#         plt.imsave(os.path.join(path4, file_name), mask1)
  


#   print("이미지 생성 중...")
#   # file_list_building = file_list_building.replace(".json",".png")
#   # file_list_road = file_list_road.replace(".json",".png")

#   for filename in file_list_building:
#       filename = filename.replace(".json",".png")
#       image = cv2.imread(os.path.join(path5, filename))
#       cv2.imwrite(os.path.join(path6, filename),image)




In [ ]:
# createFile(y_train_building_dir, y_train_road_dir, y_train_building, y_train_road, x_train_dir, x_train_building, x_train_road)
# createFile(y_valid_building_dir, y_valid_road_dir, y_valid_building, y_valid_road, x_valid_dir, x_valid_building, x_valid_road)

In [ ]:
# print(len(os.listdir(y_train_building)))
# print(len(os.listdir(y_train_road)))
# print(len(os.listdir(x_train_building)))
# print(len(os.listdir(x_train_road)))

# print(len(os.listdir(y_valid_building)))
# print(len(os.listdir(y_valid_road)))
# print(len(os.listdir(x_valid_building)))
# print(len(os.listdir(x_valid_road)))



##n분할 이미지

In [ ]:
#16분할 
# plt.imshow(a)
# plt.show() 

# import random
# from PIL import Image
# import matplotlib.pyplot as plt
# import matplotlib.image as img


# file_list = sorted(os.listdir(x_train))
# for i in range(0,800,16):
#   List = random.sample(file_list, 16)
#   images = []
#   labels = []
#   for j in List :
#     print(j)
#     image = Image.open(os.path.join(x_train, j))
#     label = Image.open(os.path.join(y_train, j))
#     for h in range(0,1024,256):
#       for w in range(0,1024,256):
#           box = (w, h, w+256, h+256)
#           a = image.crop(box)  
#           images.append(a)
#           a = label.crop(box)
#           labels.append(a)
#   for k in range(0,16):
#     filename = 'BONUS'+str(i+k)+'.png'
#     mixedImage = []
#     mixedLabel = []
#     mixedImage.append(np.hstack((np.vstack((images[k],images[k+16])),np.vstack((images[k+32],images[k+48])))))
#     mixedImage.append(np.hstack((np.vstack((images[k+64],images[k+80])),np.vstack((images[k+96],images[k+112])))))
#     mixedImage.append(np.hstack((np.vstack((images[k+128],images[k+144])),np.vstack((images[k+160],images[k+176])))))
#     mixedImage.append(np.hstack((np.vstack((images[k+192],images[k+208])),np.vstack((images[k+224],images[k+240])))))
#     image1 = np.vstack((mixedImage[0], mixedImage[1]))
#     image2 = np.vstack((mixedImage[2], mixedImage[3]))
#     image3 = np.hstack((image1,image2))
#     plt.imsave(os.path.join(x_train, filename),image3)

#     mixedLabel.append(np.hstack((np.vstack((labels[k],labels[k+16])),np.vstack((labels[k+32],labels[k+48])))))
#     mixedLabel.append(np.hstack((np.vstack((labels[k+64],labels[k+80])),np.vstack((labels[k+96],labels[k+112])))))
#     mixedLabel.append(np.hstack((np.vstack((labels[k+128],labels[k+144])),np.vstack((labels[k+160],labels[k+176])))))
#     mixedLabel.append(np.hstack((np.vstack((labels[k+192],labels[k+208])),np.vstack((labels[k+224],labels[k+240])))))
#     label1 = np.vstack((mixedLabel[0], mixedLabel[1]))
#     label2 = np.vstack((mixedLabel[2], mixedLabel[3]))
#     label3 = np.hstack((label1,label2))
#     plt.imsave(os.path.join(y_train, filename),label3)

#     # label1 = np.vstack((labels[k],labels[k+4]))
#     # label2 = np.vstack((labels[k+8],labels[k+12]))
#     # label3 = np.hstack((label1,label2))
#     # plt.imsave(os.path.join(y_train, filename),label3)
 
#     # plt.imshow(label3)
#     # plt.show()    

In [ ]:
#64분할 
import random
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as img


file_list = sorted(os.listdir(x_train))
for i in range(0,1920,64):
  List = random.sample(file_list, 64)
  images = []
  labels = []
  for j in List :
    print(j)
    image = Image.open(os.path.join(x_train, j))
    label = Image.open(os.path.join(y_train, j))
    for h in range(0,1024,128):
      for w in range(0,1024,128):
          box = (w, h, w+128, h+128)
          a = image.crop(box)  
          images.append(a)
          a = label.crop(box)
          labels.append(a)
  for k in range(0,64):
    filename = 'BONUS'+str(i+k)+'.png'
    cutedImage = []
    Image4 = []
    Image16 = []
    cutedLabel = []
    Label4 = []
    Label16 = []

    for a in range(0,4096,1024):
      Image4.clear()
      Label4.clear()
      for b in range(0, 1024, 256):
          cutedImage.clear()
          cutedLabel.clear()
          for c in range(0,193,64):
            cutedImage.append(images[c+b+a+k])
            cutedLabel.append(labels[c+b+a+k])
          n=random.sample([0,1,2,3],4)
          Image4.append(np.hstack((np.vstack((cutedImage[n[0]],cutedImage[n[1]])),np.vstack((cutedImage[n[2]],cutedImage[n[3]])))))  
          Label4.append(np.hstack((np.vstack((cutedLabel[n[0]],cutedLabel[n[1]])),np.vstack((cutedLabel[n[2]],cutedLabel[n[3]]))))) 
     
      n=random.sample([0,1,2,3],4)
      Image16.append(np.hstack((np.vstack((Image4[n[0]],Image4[n[1]])),np.vstack((Image4[n[2]],Image4[n[3]])))))
      Label16.append(np.hstack((np.vstack((Label4[n[0]],Label4[n[1]])),np.vstack((Label4[n[2]],Label4[n[3]])))))
   
    n=random.sample([0,1,2,3],4)
    image1 = np.vstack((Image16[n[0]], Image16[n[1]]))
    image2 = np.vstack((Image16[n[2]], Image16[n[3]]))
    image3 = np.hstack((image1,image2))
    plt.imsave(os.path.join(x_train, filename),image3)

    Label1 = np.vstack((Label16[n[0]], Label16[n[1]]))
    Label2 = np.vstack((Label16[n[2]], Label16[n[3]]))
    Label3 = np.hstack((Label1,Label2))
    plt.imsave(os.path.join(y_train, filename),Label3)

BLD01338_PS3_K3A_NIA0373.png
BLD09828_PS3_K3A_NIA0387.png
BLD00011_PS3_K3A_NIA0276.png
BLD01973_PS3_K3A_NIA0374.png
BLD00905_PS3_K3A_NIA0277.png
BLD00491_PS3_K3A_NIA0276.png
BLD01088_PS3_K3A_NIA0277.png
BLD02011_PS3_K3A_NIA0374.png
BLD01015_PS3_K3A_NIA0277.png
BLD01000_PS3_K3A_NIA0277.png
BLD01062_PS3_K3A_NIA0277.png
BLD01083_PS3_K3A_NIA0277.png
BLD01546_PS3_K3A_NIA0373.png
BLD03698_PS3_K3A_NIA0376.png
BLD02651_PS3_K3A_NIA0375.png
BLD00162_PS3_K3A_NIA0276.png
BLD01295_PS3_K3A_NIA0373.png
BLD02927_PS3_K3A_NIA0375.png
BLD03699_PS3_K3A_NIA0376.png
BLD00791_PS3_K3A_NIA0277.png
BLD01190_PS3_K3A_NIA0373.png
BLD01063_PS3_K3A_NIA0277.png
BLD00996_PS3_K3A_NIA0277.png
BLD03363_PS3_K3A_NIA0376.png
BLD02493_PS3_K3A_NIA0375.png
BLD00789_PS3_K3A_NIA0277.png
BLD01449_PS3_K3A_NIA0373.png
BLD02705_PS3_K3A_NIA0375.png
BLD01702_PS3_K3A_NIA0373.png
BLD02818_PS3_K3A_NIA0375.png
BLD02278_PS3_K3A_NIA0374.png
BLD00061_PS3_K3A_NIA0276.png
BLD00311_PS3_K3A_NIA0276.png
BLD02650_PS3_K3A_NIA0375.png
BLD01394_PS3_K

In [ ]:
print(len(os.listdir(x_train)))
print(len(os.listdir(y_train)))

2132
2132


## 데이터로더 정의

In [ ]:
CLASSES = ['building', 'road']

In [ ]:
class Dataset(BaseDataset):
    def __init__(
            self, 
            images_dir, 
            masks_dir, 
            classes=None, 
            augmentation=None, 
            preprocessing=None,
            mode=None,
    ):
        self.ids = os.listdir(images_dir)
        self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]
        
        # self.class_values = [255, 200]
       
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]

        for i in range(len(self.masks_fps)):
          self.mask_ids = np.unique(cv2.imread(self.masks_fps[i], 0))[1:]
          if len(self.mask_ids) == len(self.class_values):
            break
        
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.masks_fps[i],0)

        masks = [(mask == self.mask_ids[v]) for v in self.class_values]
        mask = np.stack(masks, axis=-1).astype('float')
      
        if mask.shape[-1] != 1:
            background = 1 - mask.sum(axis=-1, keepdims=True)
            mask = np.concatenate((mask, background), axis=-1)
      
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
            
        return image, mask
        
    def __len__(self):
        return len(self.ids)

## 데이터증대

In [ ]:
import albumentations as albu

In [ ]:
def get_training_augmentation():
    train_transform = [

        albu.HorizontalFlip(p=0.5),

        albu.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0),

        albu.PadIfNeeded(min_height=320, min_width=320, always_apply=True, border_mode=0),
        albu.RandomCrop(height=320, width=320, always_apply=True),

        albu.IAAAdditiveGaussianNoise(p=0.2),
        albu.IAAPerspective(p=0.5),

        albu.OneOf(
            [
                albu.CLAHE(p=1),
                albu.RandomBrightness(p=1),
                albu.RandomGamma(p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.IAASharpen(p=1),
                albu.Blur(blur_limit=3, p=1),
                albu.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),

        albu.OneOf(
            [
                albu.RandomContrast(p=1),
                albu.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),
    ]
    return albu.Compose(train_transform)


def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        albu.PadIfNeeded(384, 480)
    ]
    return albu.Compose(test_transform)


def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')


def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

In [ ]:
dataset = Dataset(x_valid, y_valid, classes=CLASSES)

image, mask = dataset[3] # get some sample
visualize(
    image=image, 
    road_mask=mask[..., 1].squeeze(),
    building_mask=mask[..., 0].squeeze(),
)

In [ ]:
augmented_dataset = Dataset(
    x_train, 
    y_train, 
    augmentation=get_training_augmentation(), 
    classes=CLASSES,
)

# same image with different random transforms
for i in range(3):
    image, mask = augmented_dataset[3]
    visualize(
        image=image, 
        road_mask=mask[..., 1].squeeze(),
        building_mask=mask[..., 0].squeeze(),
    
    )

##모델 및 encoder, activation성능 테스트

In [ ]:
def make_model(models, ENCODER, ENCODER_WEIGHTS, ACTIVATION):
    if models == "UnetPlusPlus":
        model = smp.UnetPlusPlus(
            encoder_name=ENCODER, 
            encoder_weights=ENCODER_WEIGHTS, 
            classes=len(CLASSES)+1, 
            activation=ACTIVATION,
        )
    elif models == "Unet":
        model = smp.Unet(
            encoder_name=ENCODER, 
            encoder_weights=ENCODER_WEIGHTS, 
            classes=len(CLASSES)+1, 
            activation=ACTIVATION,
        )
    elif models == "MAnet":
        model = smp.MAnet(
            encoder_name=ENCODER, 
            encoder_weights=ENCODER_WEIGHTS, 
            classes=len(CLASSES)+1, 
            activation=ACTIVATION,
        )
    elif models == "Linknet":
        model = smp.Linknet(
            encoder_name=ENCODER, 
            encoder_weights=ENCODER_WEIGHTS, 
            classes=len(CLASSES)+1, 
            activation=ACTIVATION,
        )
    elif models == "FPN":
        model = smp.FPN(
            encoder_name=ENCODER, 
            encoder_weights=ENCODER_WEIGHTS, 
            classes=len(CLASSES)+1, 
            activation=ACTIVATION,
        )
    elif models == "PSPNet":
        model = smp.PSPNet(
            encoder_name=ENCODER, 
            encoder_weights=ENCODER_WEIGHTS, 
            classes=len(CLASSES)+1, 
            activation=ACTIVATION,
        )
    elif models == "DeepLabV3":
        model = smp.DeepLabV3(
            encoder_name=ENCODER, 
            encoder_weights=ENCODER_WEIGHTS, 
            classes=len(CLASSES)+1, 
            activation=ACTIVATION,
        )
    elif models == "PAN":
        model = smp.PAN(
            encoder_name=ENCODER, 
            encoder_weights=ENCODER_WEIGHTS, 
            classes=len(CLASSES)+1, 
            activation=ACTIVATION,
        )
    elif models == "DeepLabV3Plus":
        model = smp.DeepLabV3Plus(
            encoder_name=ENCODER, 
            encoder_weights=ENCODER_WEIGHTS, 
            classes=len(CLASSES)+1, 
            activation=ACTIVATION,
        )
    return model

In [ ]:
def ModelTest(image, label, models, ENCODER, ENCODER_WEIGHTS , ACTIVATION):
    DEVICE = 'cuda'

    
#모델 설정 부분 
    model = make_model(models, ENCODER, ENCODER_WEIGHTS, ACTIVATION)
    preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

#경로지정
    valid_dataset = Dataset(
      x_valid, 
      y_valid, 
      preprocessing=get_preprocessing(preprocessing_fn),
      classes=CLASSES,
    )
    valid_loader = DataLoader(valid_dataset, batch_size=4, shuffle=False, num_workers=1)

    train_dataset = Dataset(
        image, 
        label, 
        augmentation=get_training_augmentation(), 
        preprocessing=get_preprocessing(preprocessing_fn),
        classes=CLASSES,
    )
    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)

#loss,  metrics, optimizer지정 
    loss = smp.utils.losses.DiceLoss()
    metrics = [
        smp.utils.metrics.IoU(threshold=0.5),
    ]
    optimizer = torch.optim.Adam([dict(params=model.parameters(), lr=0.001),
    ])

    train_epoch = smp.utils.train.TrainEpoch(
        model, 
        loss=loss, 
        metrics=metrics, 
        optimizer=optimizer,
        device=DEVICE,
        verbose=True,
    )

    valid_epoch = smp.utils.train.ValidEpoch(
        model, 
        loss=loss, 
        metrics=metrics, 
        device=DEVICE,
        verbose=True,
    )
#훈련 
    max_score = 0

    for i in range(0, 10):
        
        print('\nEpoch: {}'.format(i))
        train_logs = train_epoch.run(train_loader)
        valid_logs = valid_epoch.run(valid_loader)

        # do something (save model, change lr, etc.)
        if max_score < valid_logs['iou_score']:
            max_score = valid_logs['iou_score']
            print('best : ' + str(max_score))      
        

    print('max_score='+str(max_score))

In [ ]:
train_path = os.path.join(workspace_path, 'training_set')
x_train = os.path.join(train_path, 'images_테스트')
y_train = os.path.join(train_path, 'labels_테스트')

모델 성능 테스트 

In [ ]:
ModelTest(x_train, y_train,'Unet','se_resnext50_32x4d', 'imagenet' , 'softmax2d')

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
valid: 100%|██████████| 36/36 [00:24<00:00,  1.49it/s, dice_loss - 0.3361, iou_score - 0.5188]
best : 0.5187723446223472

Epoch: 1
valid: 100%|██████████| 36/36 [00:23<00:00,  1.50it/s, dice_loss - 0.2426, iou_score - 0.6239]
best : 0.623876191675663

Epoch: 2
valid: 100%|██████████| 36/36 [00:24<00:00,  1.50it/s, dice_loss - 0.2719, iou_score - 0.5842]

Epoch: 3
valid: 100%|██████████| 36/36 [00:24<00:00,  1.49it/s, dice_loss - 0.215, iou_score - 0.6538]
best : 0.6538472159041301

Epoch: 4
valid: 100%|██████████| 36/36 [00:24<00:00,  1.50it/s, dice_loss - 0.2799, iou_score - 0.5703]

Epoch: 5
valid: 100%|██████████| 36/36 [00:24<00:00,  1.49it/s, dice_loss - 0.2226, iou_score - 0.6446]

Epoch: 6
valid: 100%|██████████| 36/36 [00:23<00:00,  1.50it/s, dice_loss - 0.2314, iou_score - 0.6326]

Epoch: 7
valid: 100%|██████████| 36/36 [00:24<00:00,  1.50it/s, dice_loss - 0.225, iou_score - 0.6459]

Epoch: 8
valid: 100%|██████████| 36/36 [00:24<00:00,  1.49it/s, dice_loss - 0.2214, 

In [ ]:
ModelTest(x_train, y_train,'UnetPlusPlus','se_resnext50_32x4d', 'imagenet' , 'softmax2d')

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
valid: 100%|██████████| 36/36 [00:32<00:00,  1.12it/s, dice_loss - 0.4405, iou_score - 0.4014]
best : 0.4013595167133543

Epoch: 1
valid: 100%|██████████| 36/36 [00:32<00:00,  1.12it/s, dice_loss - 0.3159, iou_score - 0.5347]
best : 0.5346798416641023

Epoch: 2
valid: 100%|██████████| 36/36 [00:32<00:00,  1.12it/s, dice_loss - 0.2282, iou_score - 0.6386]
best : 0.6385739983783828

Epoch: 3
valid: 100%|██████████| 36/36 [00:32<00:00,  1.12it/s, dice_loss - 0.2059, iou_score - 0.6657]
best : 0.6657087422079511

Epoch: 4
valid: 100%|██████████| 36/36 [00:32<00:00,  1.12it/s, dice_loss - 0.2322, iou_score - 0.6301]

Epoch: 5
valid: 100%|██████████| 36/36 [00:32<00:00,  1.12it/s, dice_loss - 0.2137, iou_score - 0.6546]

Epoch: 6
valid: 100%|██████████| 36/36 [00:32<00:00,  1.12it/s, dice_loss - 0.2526, iou_score - 0.6068]

Epoch: 7
valid: 100%|██████████| 36/36 [00:32<00:00,  1.12it/s, dice_loss - 0.2284, iou_score - 0.6356]

Epoch: 8
valid: 100%|██████████| 36/36 [00:32<00:00,  1

In [ ]:
ModelTest(x_train, y_train,'MAnet','se_resnext50_32x4d', 'imagenet' , 'softmax2d')

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
valid: 100%|██████████| 36/36 [00:26<00:00,  1.34it/s, dice_loss - 0.2697, iou_score - 0.5955]
best : 0.5954864770174026

Epoch: 1
valid: 100%|██████████| 36/36 [00:27<00:00,  1.33it/s, dice_loss - 0.2473, iou_score - 0.6165]
best : 0.6165048703551292

Epoch: 2
valid: 100%|██████████| 36/36 [00:27<00:00,  1.32it/s, dice_loss - 0.2759, iou_score - 0.5806]

Epoch: 3
valid: 100%|██████████| 36/36 [00:27<00:00,  1.33it/s, dice_loss - 0.2146, iou_score - 0.6556]
best : 0.6556471776631143

Epoch: 4
valid: 100%|██████████| 36/36 [00:27<00:00,  1.33it/s, dice_loss - 0.2217, iou_score - 0.6484]

Epoch: 5
valid: 100%|██████████| 36/36 [00:27<00:00,  1.32it/s, dice_loss - 0.2336, iou_score - 0.6294]

Epoch: 6
valid: 100%|██████████| 36/36 [00:26<00:00,  1.33it/s, dice_loss - 0.2291, iou_score - 0.6326]

Epoch: 7
valid: 100%|██████████| 36/36 [00:27<00:00,  1.32it/s, dice_loss - 0.2217, iou_score - 0.6454]

Epoch: 8
valid: 100%|██████████| 36/36 [00:27<00:00,  1.33it/s, dice_loss - 0.355

In [ ]:
ModelTest(x_train, y_train,'Linknet','se_resnext50_32x4d', 'imagenet' , 'softmax2d')

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
valid: 100%|██████████| 36/36 [00:26<00:00,  1.36it/s, dice_loss - 0.4094, iou_score - 0.4578]
best : 0.45780552013052833

Epoch: 1
valid: 100%|██████████| 36/36 [00:26<00:00,  1.36it/s, dice_loss - 0.2548, iou_score - 0.6271]
best : 0.6270672637555333

Epoch: 2
valid: 100%|██████████| 36/36 [00:26<00:00,  1.35it/s, dice_loss - 0.2326, iou_score - 0.6438]
best : 0.6438311139742534

Epoch: 3
valid: 100%|██████████| 36/36 [00:26<00:00,  1.34it/s, dice_loss - 0.2162, iou_score - 0.6589]
best : 0.6589275035593243

Epoch: 4
valid: 100%|██████████| 36/36 [00:26<00:00,  1.35it/s, dice_loss - 0.2215, iou_score - 0.6495]

Epoch: 5
valid: 100%|██████████| 36/36 [00:26<00:00,  1.34it/s, dice_loss - 0.2382, iou_score - 0.6234]

Epoch: 6
valid: 100%|██████████| 36/36 [00:26<00:00,  1.34it/s, dice_loss - 0.2166, iou_score - 0.6527]

Epoch: 7
valid: 100%|██████████| 36/36 [00:26<00:00,  1.34it/s, dice_loss - 0.2159, iou_score - 0.6534]

Epoch: 8
valid: 100%|██████████| 36/36 [00:26<00:00,  

In [ ]:
ModelTest(x_train, y_train,'FPN','se_resnext50_32x4d', 'imagenet' , 'softmax2d')

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
valid: 100%|██████████| 36/36 [00:26<00:00,  1.38it/s, dice_loss - 0.3283, iou_score - 0.5233]
best : 0.5233410803808106

Epoch: 1
valid: 100%|██████████| 36/36 [00:26<00:00,  1.38it/s, dice_loss - 0.3283, iou_score - 0.5233]

Epoch: 2
valid: 100%|██████████| 36/36 [00:26<00:00,  1.38it/s, dice_loss - 0.3283, iou_score - 0.5233]

Epoch: 3
valid: 100%|██████████| 36/36 [00:26<00:00,  1.36it/s, dice_loss - 0.3283, iou_score - 0.5233]

Epoch: 4
valid: 100%|██████████| 36/36 [00:26<00:00,  1.37it/s, dice_loss - 0.3283, iou_score - 0.5233]

Epoch: 5
valid: 100%|██████████| 36/36 [00:26<00:00,  1.35it/s, dice_loss - 0.3283, iou_score - 0.5233]

Epoch: 6
valid: 100%|██████████| 36/36 [00:26<00:00,  1.37it/s, dice_loss - 0.3283, iou_score - 0.5233]

Epoch: 7
valid: 100%|██████████| 36/36 [00:26<00:00,  1.35it/s, dice_loss - 0.3283, iou_score - 0.5233]

Epoch: 8
valid: 100%|██████████| 36/36 [00:26<00:00,  1.37it/s, dice_loss - 0.3283, iou_score - 0.5233]

Epoch: 9
valid: 100%|███████

In [ ]:
ModelTest(x_train, y_train,'PSPNet','se_resnext50_32x4d', 'imagenet' , 'softmax2d')

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
valid: 100%|██████████| 36/36 [00:25<00:00,  1.41it/s, dice_loss - 0.3283, iou_score - 0.5233]
best : 0.5233410803808106

Epoch: 1
valid: 100%|██████████| 36/36 [00:25<00:00,  1.44it/s, dice_loss - 0.3283, iou_score - 0.5233]

Epoch: 2
valid: 100%|██████████| 36/36 [00:25<00:00,  1.42it/s, dice_loss - 0.3283, iou_score - 0.5233]

Epoch: 3
valid: 100%|██████████| 36/36 [00:25<00:00,  1.43it/s, dice_loss - 0.3283, iou_score - 0.5233]

Epoch: 4
valid: 100%|██████████| 36/36 [00:25<00:00,  1.44it/s, dice_loss - 0.3283, iou_score - 0.5233]

Epoch: 5
valid: 100%|██████████| 36/36 [00:25<00:00,  1.44it/s, dice_loss - 0.3283, iou_score - 0.5233]

Epoch: 6
valid: 100%|██████████| 36/36 [00:25<00:00,  1.43it/s, dice_loss - 0.3283, iou_score - 0.5233]

Epoch: 7
valid: 100%|██████████| 36/36 [00:25<00:00,  1.44it/s, dice_loss - 0.3282, iou_score - 0.5233]

Epoch: 8
valid: 100%|██████████| 36/36 [00:25<00:00,  1.43it/s, dice_loss - 0.3283, iou_score - 0.5233]

Epoch: 9
valid: 100%|███████

In [ ]:
ModelTest(x_train, y_train,'PAN','se_resnext50_32x4d', 'imagenet' , 'softmax2d')

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
valid: 100%|██████████| 36/36 [00:27<00:00,  1.31it/s, dice_loss - 0.3305, iou_score - 0.5242]
best : 0.5241538476612833

Epoch: 1
valid: 100%|██████████| 36/36 [00:27<00:00,  1.30it/s, dice_loss - 0.2568, iou_score - 0.6117]
best : 0.611701824598842

Epoch: 2
valid: 100%|██████████| 36/36 [00:27<00:00,  1.30it/s, dice_loss - 0.2725, iou_score - 0.5839]

Epoch: 3
valid: 100%|██████████| 36/36 [00:27<00:00,  1.30it/s, dice_loss - 0.2707, iou_score - 0.5877]

Epoch: 4
valid: 100%|██████████| 36/36 [00:27<00:00,  1.30it/s, dice_loss - 0.2427, iou_score - 0.6207]
best : 0.6206749172674285

Epoch: 5
valid: 100%|██████████| 36/36 [00:27<00:00,  1.30it/s, dice_loss - 0.2187, iou_score - 0.6508]
best : 0.6508209771580168

Epoch: 6
valid: 100%|██████████| 36/36 [00:27<00:00,  1.30it/s, dice_loss - 0.2361, iou_score - 0.6278]

Epoch: 7
valid: 100%|██████████| 36/36 [00:27<00:00,  1.30it/s, dice_loss - 0.2209, iou_score - 0.6466]

Epoch: 8
valid: 100%|██████████| 36/36 [00:27<00:00,  1.

In [ ]:
ModelTest(x_train, y_train,'DeepLabV3','se_resnext50_32x4d', 'imagenet' , 'softmax2d')

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
valid: 100%|██████████| 36/36 [00:57<00:00,  1.61s/it, dice_loss - 0.3322, iou_score - 0.5236]
best : 0.5235939365294244

Epoch: 1
valid: 100%|██████████| 36/36 [00:57<00:00,  1.60s/it, dice_loss - 0.3194, iou_score - 0.5329]
best : 0.5329127767019801

Epoch: 2
valid: 100%|██████████| 36/36 [00:57<00:00,  1.60s/it, dice_loss - 0.2795, iou_score - 0.5765]
best : 0.5764780375692581

Epoch: 3
valid: 100%|██████████| 36/36 [00:57<00:00,  1.60s/it, dice_loss - 0.2681, iou_score - 0.5905]
best : 0.5905307191941475

Epoch: 4
valid: 100%|██████████| 36/36 [00:57<00:00,  1.60s/it, dice_loss - 0.2695, iou_score - 0.5885]

Epoch: 5
valid: 100%|██████████| 36/36 [00:57<00:00,  1.60s/it, dice_loss - 0.2471, iou_score - 0.6144]
best : 0.6143681638770632

Epoch: 6
valid: 100%|██████████| 36/36 [00:57<00:00,  1.60s/it, dice_loss - 0.2412, iou_score - 0.6217]
best : 0.6216521221730444

Epoch: 7
valid: 100%|██████████| 36/36 [00:57<00:00,  1.60s/it, dice_loss - 0.2565, iou_score - 0.6027]

Epo

In [ ]:
ModelTest(x_train, y_train,'DeepLabV3Plus','se_resnext50_32x4d', 'imagenet' , 'softmax2d')

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
valid: 100%|██████████| 36/36 [00:28<00:00,  1.28it/s, dice_loss - 0.3351, iou_score - 0.533]
best : 0.5330455642607476

Epoch: 1
valid: 100%|██████████| 36/36 [00:27<00:00,  1.30it/s, dice_loss - 0.2958, iou_score - 0.5596]
best : 0.5595531455344623

Epoch: 2
valid: 100%|██████████| 36/36 [00:28<00:00,  1.27it/s, dice_loss - 0.4022, iou_score - 0.4382]

Epoch: 3
valid: 100%|██████████| 36/36 [00:27<00:00,  1.29it/s, dice_loss - 0.2696, iou_score - 0.5865]
best : 0.5864667511648604

Epoch: 4
valid: 100%|██████████| 36/36 [00:27<00:00,  1.29it/s, dice_loss - 0.3283, iou_score - 0.5233]

Epoch: 5
valid: 100%|██████████| 36/36 [00:28<00:00,  1.28it/s, dice_loss - 0.2655, iou_score - 0.5926]
best : 0.5926277678873805

Epoch: 6
valid: 100%|██████████| 36/36 [00:28<00:00,  1.28it/s, dice_loss - 0.2469, iou_score - 0.6128]
best : 0.6128304344084529

Epoch: 7
valid: 100%|██████████| 36/36 [00:27<00:00,  1.29it/s, dice_loss - 0.3101, iou_score - 0.5424]

Epoch: 8
valid: 100%|█████████

In [ ]:
ModelTest(x_train, y_train,'UnetPlusPlus','se_resnext50_32x4d', 'imagenet' , 'sigmoid')

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - 0.2572, iou_score - 0.6396]
best : 0.639600407746103

Epoch: 1
valid: 100%|██████████| 36/36 [00:32<00:00,  1.10it/s, dice_loss - 0.3019, iou_score - 0.56]

Epoch: 2
valid: 100%|██████████| 36/36 [00:32<00:00,  1.10it/s, dice_loss - 0.2287, iou_score - 0.6426]
best : 0.642649359173245

Epoch: 3
valid: 100%|██████████| 36/36 [00:32<00:00,  1.10it/s, dice_loss - 0.2536, iou_score - 0.601]

Epoch: 4
valid: 100%|██████████| 36/36 [00:32<00:00,  1.10it/s, dice_loss - 0.2887, iou_score - 0.5582]

Epoch: 5
valid: 100%|██████████| 36/36 [00:32<00:00,  1.10it/s, dice_loss - 0.2272, iou_score - 0.6373]

Epoch: 6
valid: 100%|██████████| 36/36 [00:32<00:00,  1.10it/s, dice_loss - 0.2154, iou_score - 0.6525]
best : 0.6525370842880671

Epoch: 7
valid: 100%|██████████| 36/36 [00:32<00:00,  1.10it/s, dice_loss - 0.2106, iou_score - 0.6596]
best : 0.6596267811126179

Epoch: 8
valid: 100%|██████████| 36/36 [00:32<00:00,  1.10it

In [ ]:
ModelTest(x_train, y_train,'UnetPlusPlus','efficientnet-b7', 'imagenet' , 'softmax2d')

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
valid: 100%|██████████| 36/36 [00:29<00:00,  1.23it/s, dice_loss - 0.2791, iou_score - 0.5845]
best : 0.5844741380876964

Epoch: 1
valid: 100%|██████████| 36/36 [00:29<00:00,  1.23it/s, dice_loss - 0.2236, iou_score - 0.6473]
best : 0.6473044587506186

Epoch: 2
valid: 100%|██████████| 36/36 [00:29<00:00,  1.23it/s, dice_loss - 0.2259, iou_score - 0.64]

Epoch: 3
valid: 100%|██████████| 36/36 [00:29<00:00,  1.23it/s, dice_loss - 0.2271, iou_score - 0.6392]

Epoch: 4
valid: 100%|██████████| 36/36 [00:29<00:00,  1.23it/s, dice_loss - 0.2063, iou_score - 0.665]
best : 0.6649582286675774

Epoch: 5
valid: 100%|██████████| 36/36 [00:29<00:00,  1.23it/s, dice_loss - 0.2244, iou_score - 0.6401]

Epoch: 6
valid: 100%|██████████| 36/36 [00:29<00:00,  1.23it/s, dice_loss - 0.235, iou_score - 0.6276]

Epoch: 7
valid: 100%|██████████| 36/36 [00:29<00:00,  1.23it/s, dice_loss - 0.2312, iou_score - 0.6327]

Epoch: 8
valid: 100%|██████████| 36/36 [00:29<00:00,  1.23it/s, dice_loss - 0.2145, i

In [ ]:
ModelTest(x_train, y_train,'UnetPlusPlus','efficientnet-b5', 'imagenet' , 'softmax2d')

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b5-b6417697.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b5-b6417697.pth


  0%|          | 0.00/117M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
valid: 100%|██████████| 36/36 [00:25<00:00,  1.39it/s, dice_loss - 0.3338, iou_score - 0.5211]
best : 0.5211327100793521

Epoch: 1
valid: 100%|██████████| 36/36 [00:25<00:00,  1.39it/s, dice_loss - 0.3554, iou_score - 0.4857]

Epoch: 2
valid: 100%|██████████| 36/36 [00:25<00:00,  1.40it/s, dice_loss - 0.3332, iou_score - 0.5096]

Epoch: 3
valid: 100%|██████████| 36/36 [00:25<00:00,  1.39it/s, dice_loss - 0.2091, iou_score - 0.6617]
best : 0.6617498546838761

Epoch: 4
valid: 100%|██████████| 36/36 [00:25<00:00,  1.40it/s, dice_loss - 0.2008, iou_score - 0.6725]
best : 0.6725490109788046

Epoch: 5
valid: 100%|██████████| 36/36 [00:25<00:00,  1.39it/s, dice_loss - 0.2338, iou_score - 0.6291]

Epoch: 6
valid: 100%|██████████| 36/36 [00:26<00:00,  1.38it/s, dice_loss - 0.2248, iou_score - 0.6402]

Epoch: 7
valid: 100%|██████████| 36/36 [00:25<00:00,  1.39it/s, dice_loss - 0.2583, iou_score - 0.5981]

Epoch: 8
valid: 100%|██████████| 36/36 [00:25<00:00,  1.39it/s, dice_loss - 0.199

In [ ]:
ModelTest(x_train, y_train,'UnetPlusPlus','efficientnet-b3', 'imagenet' , 'softmax2d')

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b3-5fb5a3c3.pth


  0%|          | 0.00/47.1M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
valid: 100%|██████████| 36/36 [00:24<00:00,  1.49it/s, dice_loss - 0.3664, iou_score - 0.4857]
best : 0.4856642575727569

Epoch: 1
valid: 100%|██████████| 36/36 [00:24<00:00,  1.48it/s, dice_loss - 0.2499, iou_score - 0.6135]
best : 0.6134607783622212

Epoch: 2
valid: 100%|██████████| 36/36 [00:24<00:00,  1.46it/s, dice_loss - 0.3139, iou_score - 0.5315]

Epoch: 3
valid: 100%|██████████| 36/36 [00:24<00:00,  1.48it/s, dice_loss - 0.2726, iou_score - 0.5811]

Epoch: 4
valid: 100%|██████████| 36/36 [00:24<00:00,  1.47it/s, dice_loss - 0.2072, iou_score - 0.665]
best : 0.6649837833311822

Epoch: 5
valid: 100%|██████████| 36/36 [00:24<00:00,  1.47it/s, dice_loss - 0.1994, iou_score - 0.6741]
best : 0.6741313106483882

Epoch: 6
valid: 100%|██████████| 36/36 [00:24<00:00,  1.48it/s, dice_loss - 0.2081, iou_score - 0.6625]

Epoch: 7
valid: 100%|██████████| 36/36 [00:24<00:00,  1.47it/s, dice_loss - 0.5014, iou_score - 0.3456]

Epoch: 8
valid: 100%|██████████| 36/36 [00:24<00:00,  1.

In [ ]:
ModelTest(x_train, y_train,'UnetPlusPlus','resnet34', 'imagenet' , 'softmax2d')

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
valid: 100%|██████████| 36/36 [00:23<00:00,  1.50it/s, dice_loss - 0.3247, iou_score - 0.5233]
best : 0.5233407103353076

Epoch: 1
valid: 100%|██████████| 36/36 [00:23<00:00,  1.51it/s, dice_loss - 0.3156, iou_score - 0.5233]
best : 0.5233490847878987

Epoch: 2
valid: 100%|██████████| 36/36 [00:23<00:00,  1.51it/s, dice_loss - 0.2828, iou_score - 0.5741]
best : 0.5741104947196112

Epoch: 3
valid: 100%|██████████| 36/36 [00:24<00:00,  1.49it/s, dice_loss - 0.4423, iou_score - 0.388]

Epoch: 4
valid: 100%|██████████| 36/36 [00:24<00:00,  1.49it/s, dice_loss - 0.2759, iou_score - 0.576]
best : 0.5759562320179408

Epoch: 5
valid: 100%|██████████| 36/36 [00:24<00:00,  1.49it/s, dice_loss - 0.2778, iou_score - 0.5776]
best : 0.5775610067778164

Epoch: 6
valid: 100%|██████████| 36/36 [00:24<00:00,  1.49it/s, dice_loss - 0.2677, iou_score - 0.591]
best : 0.5910062218705813

Epoch: 7
valid: 100%|██████████| 36/36 [00:24<00:00,  1.49it/s, dice_loss - 0.2735, iou_score - 0.5786]

Epoch:

In [ ]:
ModelTest(x_train, y_train,'UnetPlusPlus','resnet101', 'imagenet' , 'softmax2d')

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/hub/checkpoints/resnet101-5d3b4d8f.pth


  0%|          | 0.00/170M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
valid: 100%|██████████| 36/36 [00:29<00:00,  1.20it/s, dice_loss - 0.3444, iou_score - 0.5233]
best : 0.5233410754137569

Epoch: 1
valid: 100%|██████████| 36/36 [00:30<00:00,  1.20it/s, dice_loss - 0.326, iou_score - 0.5233]
best : 0.5233410803808106

Epoch: 2
valid: 100%|██████████| 36/36 [00:29<00:00,  1.20it/s, dice_loss - 0.3321, iou_score - 0.5233]
best : 0.5233411714434625

Epoch: 3
valid: 100%|██████████| 36/36 [00:30<00:00,  1.20it/s, dice_loss - 0.3284, iou_score - 0.5233]

Epoch: 4
valid: 100%|██████████| 36/36 [00:30<00:00,  1.20it/s, dice_loss - 0.3216, iou_score - 0.5233]

Epoch: 5
valid: 100%|██████████| 36/36 [00:30<00:00,  1.20it/s, dice_loss - 0.2854, iou_score - 0.5747]
best : 0.5746508464217187

Epoch: 6
valid: 100%|██████████| 36/36 [00:30<00:00,  1.19it/s, dice_loss - 0.2817, iou_score - 0.5758]
best : 0.575755212869909

Epoch: 7
valid: 100%|██████████| 36/36 [00:30<00:00,  1.20it/s, dice_loss - 0.3061, iou_score - 0.5477]

Epoch: 8
valid: 100%|██████████

In [ ]:
ModelTest(x_train, y_train,'UnetPlusPlus','vgg16', 'imagenet' , 'softmax2d')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
valid: 100%|██████████| 36/36 [01:51<00:00,  3.09s/it, dice_loss - 0.3459, iou_score - 0.529]
best : 0.5290493360824055

Epoch: 1
valid: 100%|██████████| 36/36 [00:37<00:00,  1.05s/it, dice_loss - 0.2878, iou_score - 0.5666]
best : 0.5665998227066464

Epoch: 2
valid: 100%|██████████| 36/36 [00:38<00:00,  1.06s/it, dice_loss - 0.3019, iou_score - 0.5516]

Epoch: 3
valid: 100%|██████████| 36/36 [00:38<00:00,  1.06s/it, dice_loss - 0.3028, iou_score - 0.5508]

Epoch: 4
valid: 100%|██████████| 36/36 [00:37<00:00,  1.05s/it, dice_loss - 0.3149, iou_score - 0.5388]

Epoch: 5
valid: 100%|██████████| 36/36 [00:37<00:00,  1.05s/it, dice_loss - 0.2954, iou_score - 0.5563]

Epoch: 6
valid: 100%|██████████| 36/36 [00:37<00:00,  1.05s/it, dice_loss - 0.2994, iou_score - 0.5541]

Epoch: 7
valid: 100%|██████████| 36/36 [00:37<00:00,  1.06s/it, dice_loss - 0.3012, iou_score - 0.5496]

Epoch: 8
valid: 100%|██████████| 36/36 [00:37<00:00,  1.05s/it, dice_loss - 0.2888, iou_score - 0.5641]

Epo

In [ ]:
ModelTest(x_train, y_train,'UnetPlusPlus','se_resnext101_32x4d', 'imagenet' , 'softmax2d')

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/se_resnext101_32x4d-3b2fe3d8.pth" to /root/.cache/torch/hub/checkpoints/se_resnext101_32x4d-3b2fe3d8.pth


  0%|          | 0.00/187M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
valid: 100%|██████████| 36/36 [00:41<00:00,  1.15s/it, dice_loss - 0.2841, iou_score - 0.6168]
best : 0.6167861206663979

Epoch: 1
valid: 100%|██████████| 36/36 [00:41<00:00,  1.16s/it, dice_loss - 0.2725, iou_score - 0.5884]

Epoch: 2
valid: 100%|██████████| 36/36 [00:41<00:00,  1.16s/it, dice_loss - 0.2728, iou_score - 0.5947]

Epoch: 3
valid: 100%|██████████| 36/36 [00:41<00:00,  1.16s/it, dice_loss - 0.4131, iou_score - 0.4221]

Epoch: 4
valid: 100%|██████████| 36/36 [00:41<00:00,  1.16s/it, dice_loss - 0.2202, iou_score - 0.6469]
best : 0.6469173365169099

Epoch: 5
valid: 100%|██████████| 36/36 [00:41<00:00,  1.16s/it, dice_loss - 0.2039, iou_score - 0.6698]
best : 0.6698164617021878

Epoch: 6
valid: 100%|██████████| 36/36 [00:41<00:00,  1.16s/it, dice_loss - 0.2138, iou_score - 0.6544]

Epoch: 7
valid: 100%|██████████| 36/36 [00:41<00:00,  1.16s/it, dice_loss - 0.2307, iou_score - 0.6324]

Epoch: 8
valid: 100%|██████████| 36/36 [00:41<00:00,  1.16s/it, dice_loss - 0.239

In [ ]:
ModelTest(x_train, y_train,'UnetPlusPlus','se_resnext50_32x4d', 'imagenet' , 'softmax')

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
train:   0%|          | 0/53 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/segmentation_models_pytorch/base/modules.py:115: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.activation(x)


valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - 0.399, iou_score - 0.4477]
best : 0.447662101023727

Epoch: 1
valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - 0.2237, iou_score - 0.6457]
best : 0.6456653103232383

Epoch: 2
valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - 0.2276, iou_score - 0.6382]

Epoch: 3
valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - 0.2249, iou_score - 0.6402]

Epoch: 4
valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - 0.2386, iou_score - 0.623]

Epoch: 5
valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - 0.2477, iou_score - 0.6136]

Epoch: 6
valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - 0.2141, iou_score - 0.6544]
best : 0.6544295797745386

Epoch: 7
valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - 0.2183, iou_score - 0.649]

Epoch: 8
valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - 0.2195, iou_score -

callable/sigmoid/softmax/logsoftmax/tanh/None;

In [ ]:
ModelTest(x_train, y_train,'UnetPlusPlus','se_resnext50_32x4d', 'imagenet' , 'logsoftmax')

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
train:   0%|          | 0/53 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/segmentation_models_pytorch/base/modules.py:115: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.activation(x)


valid: 100%|██████████| 36/36 [00:31<00:00,  1.13it/s, dice_loss - -0.5873, iou_score - 2.45e-14]
best : 2.4504132027351594e-14

Epoch: 1
valid: 100%|██████████| 36/36 [00:32<00:00,  1.12it/s, dice_loss - -0.6934, iou_score - 2.45e-14]

Epoch: 2
valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - -0.7911, iou_score - 2.45e-14]

Epoch: 3
valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - -0.6486, iou_score - 2.45e-14]

Epoch: 4
valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - -0.8194, iou_score - 2.45e-14]

Epoch: 5
valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - -0.7926, iou_score - 2.45e-14]

Epoch: 6
valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - -0.7649, iou_score - 2.45e-14]

Epoch: 7
valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - -0.8156, iou_score - 2.45e-14]

Epoch: 8
valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - -0.8646, iou_score - 2.45e-14]

Epoch: 

In [ ]:
ModelTest(x_train, y_train,'UnetPlusPlus','se_resnext50_32x4d', 'imagenet' , 'tanh')

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
valid: 100%|██████████| 36/36 [00:31<00:00,  1.13it/s, dice_loss - 0.7054, iou_score - 0.2366]
best : 0.23662879773312143

Epoch: 1
valid: 100%|██████████| 36/36 [00:32<00:00,  1.12it/s, dice_loss - 2.107, iou_score - 0.3625]
best : 0.36252993862662053

Epoch: 2
valid: 100%|██████████| 36/36 [00:32<00:00,  1.12it/s, dice_loss - 5.05, iou_score - 0.1743]

Epoch: 3
valid: 100%|██████████| 36/36 [00:32<00:00,  1.12it/s, dice_loss - 0.5788, iou_score - 0.09136]

Epoch: 4
valid: 100%|██████████| 36/36 [00:32<00:00,  1.12it/s, dice_loss - 0.4746, iou_score - 0.03064]

Epoch: 5
valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - 0.2855, iou_score - 0.01545]

Epoch: 6
valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - 0.1838, iou_score - 0.008953]

Epoch: 7
valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - 0.02488, iou_score - 0.006985]

Epoch: 8
valid: 100%|██████████| 36/36 [00:32<00:00,  1.12it/s, dice_loss - -0.2243, iou_score - 0.0

In [ ]:
ModelTest(x_train, y_train,'UnetPlusPlus','se_resnext50_32x4d', 'imagenet' , 'sigmoid')

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - 0.288, iou_score - 0.5862]
best : 0.5862023755908014

Epoch: 1
valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - 0.2638, iou_score - 0.5982]
best : 0.5982326459553506

Epoch: 2
valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - 0.2578, iou_score - 0.6091]
best : 0.6090780562824673

Epoch: 3
valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - 0.245, iou_score - 0.6164]
best : 0.616415290368928

Epoch: 4
valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - 0.2407, iou_score - 0.6207]
best : 0.6206536201967133

Epoch: 5
valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - 0.2489, iou_score - 0.6033]

Epoch: 6
valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - 0.2087, iou_score - 0.6647]
best : 0.6646871947579914

Epoch: 7
valid: 100%|██████████| 36/36 [00:32<00:00,  1.11it/s, dice_loss - 0.2207, iou_score - 0.6479]

Epoch:

In [ ]:
ModelTest(x_train, y_train,'UnetPlusPlus','efficientnet-b7', 'imagenet' , 'softmax')

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b7-dcc49843.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b7-dcc49843.pth


  0%|          | 0.00/254M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
train:   0%|          | 0/53 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/segmentation_models_pytorch/base/modules.py:115: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.activation(x)


valid: 100%|██████████| 36/36 [00:29<00:00,  1.24it/s, dice_loss - 0.6796, iou_score - 0.1931]
best : 0.19305303527249235

Epoch: 1
valid: 100%|██████████| 36/36 [00:29<00:00,  1.24it/s, dice_loss - 0.3713, iou_score - 0.4643]
best : 0.46434401224056887

Epoch: 2
valid: 100%|██████████| 36/36 [00:29<00:00,  1.24it/s, dice_loss - 0.4169, iou_score - 0.4177]

Epoch: 3
valid: 100%|██████████| 36/36 [00:29<00:00,  1.24it/s, dice_loss - 0.2077, iou_score - 0.6633]
best : 0.6633003767993713

Epoch: 4
valid: 100%|██████████| 36/36 [00:29<00:00,  1.23it/s, dice_loss - 0.2197, iou_score - 0.6477]

Epoch: 5
valid: 100%|██████████| 36/36 [00:29<00:00,  1.23it/s, dice_loss - 0.2279, iou_score - 0.6379]

Epoch: 6
valid: 100%|██████████| 36/36 [00:29<00:00,  1.23it/s, dice_loss - 0.208, iou_score - 0.6619]

Epoch: 7
valid: 100%|██████████| 36/36 [00:29<00:00,  1.23it/s, dice_loss - 0.2036, iou_score - 0.6669]
best : 0.6669099811050627

Epoch: 8
valid: 100%|██████████| 36/36 [00:29<00:00,  1.23it/s, 

In [ ]:
ModelTest(x_train, y_train,'UnetPlusPlus','se_resnext101_32x4d', 'imagenet' , 'softmax')

Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/se_resnext101_32x4d-3b2fe3d8.pth" to /root/.cache/torch/hub/checkpoints/se_resnext101_32x4d-3b2fe3d8.pth


  0%|          | 0.00/187M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
train:   0%|          | 0/53 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/segmentation_models_pytorch/base/modules.py:115: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.activation(x)


valid: 100%|██████████| 36/36 [00:41<00:00,  1.15s/it, dice_loss - 0.4533, iou_score - 0.3884]
best : 0.38837263195051086

Epoch: 1
valid: 100%|██████████| 36/36 [00:41<00:00,  1.16s/it, dice_loss - 0.2472, iou_score - 0.6158]
best : 0.6157604058583576

Epoch: 2
valid: 100%|██████████| 36/36 [00:41<00:00,  1.16s/it, dice_loss - 0.2292, iou_score - 0.6379]
best : 0.6379425765739546

Epoch: 3
valid: 100%|██████████| 36/36 [00:41<00:00,  1.16s/it, dice_loss - 0.2283, iou_score - 0.6375]

Epoch: 4
valid: 100%|██████████| 36/36 [00:41<00:00,  1.17s/it, dice_loss - 0.2443, iou_score - 0.6149]

Epoch: 5
valid: 100%|██████████| 36/36 [00:41<00:00,  1.17s/it, dice_loss - 0.2268, iou_score - 0.6389]
best : 0.6388994314604335

Epoch: 6
valid: 100%|██████████| 36/36 [00:42<00:00,  1.17s/it, dice_loss - 0.2065, iou_score - 0.6642]
best : 0.6641898014479211

Epoch: 7
valid: 100%|██████████| 36/36 [00:41<00:00,  1.17s/it, dice_loss - 0.222, iou_score - 0.645]

Epoch: 8
valid: 100%|██████████| 36/36 [

##데이터에 따른 학습 테스트

In [ ]:
def UnetPlusPlus_ModelTest(image, label, ENCODER, ENCODER_WEIGHTS , ACTIVATION):
   
    DEVICE = 'cuda'
 
#모델 설정 부분 
    model = smp.UnetPlusPlus(
        encoder_name=ENCODER, 
        encoder_weights=ENCODER_WEIGHTS, 
        classes=len(CLASSES)+1, 
        activation=ACTIVATION,
    )
    preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

#경로지정
    valid_dataset = Dataset(
      x_valid, 
      y_valid, 
      preprocessing=get_preprocessing(preprocessing_fn),
      classes=CLASSES,
    )
    valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=1)

    train_dataset = Dataset(
        image, 
        label, 
        augmentation=get_training_augmentation(), 
        preprocessing=get_preprocessing(preprocessing_fn),
        classes=CLASSES,
    )
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)

#loss,  metrics, optimizer지정 
    loss = smp.utils.losses.DiceLoss()
    metrics = [
        smp.utils.metrics.IoU(threshold=0.5),
    ]
    optimizer = torch.optim.Adam([dict(params=model.parameters(), lr=0.001),
    ])
    train_epoch = smp.utils.train.TrainEpoch(
        model, 
        loss=loss, 
        metrics=metrics, 
        optimizer=optimizer,
        device=DEVICE,
        verbose=True,
    )

    valid_epoch = smp.utils.train.ValidEpoch(
        model, 
        loss=loss, 
        metrics=metrics, 
        device=DEVICE,
        verbose=True,
    )
    max_score = 0
    for i in range(0, 10):    
        print('\nEpoch: {}'.format(i))
        train_logs = train_epoch.run(train_loader)
        valid_logs = valid_epoch.run(valid_loader)

        if max_score < valid_logs['iou_score']:
            max_score = valid_logs['iou_score']
            print('best : ' + str(max_score))      
        
    print('max_score='+str(max_score))

In [ ]:
#기본으로 제공된 이미지에 대한 훈련
x_train = os.path.join(train_path, 'images_원본')
y_train = os.path.join(train_path, 'labels_원본')
UnetPlusPlus_ModelTest(x_train, y_train,'se_resnext50_32x4d', 'imagenet' , 'softmax2d')

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
valid: 100%|██████████| 142/142 [00:35<00:00,  4.01it/s, dice_loss - 0.3834, iou_score - 0.4417]
best : 0.44166825779936686

Epoch: 1
valid: 100%|██████████| 142/142 [00:35<00:00,  4.05it/s, dice_loss - 0.2153, iou_score - 0.6701]
best : 0.6701442834357142

Epoch: 2
valid: 100%|██████████| 142/142 [00:35<00:00,  4.01it/s, dice_loss - 0.2031, iou_score - 0.6848]
best : 0.6848391651267735

Epoch: 3
valid: 100%|██████████| 142/142 [00:35<00:00,  3.96it/s, dice_loss - 0.2076, iou_score - 0.6764]

Epoch: 4
valid: 100%|██████████| 142/142 [00:35<00:00,  4.00it/s, dice_loss - 0.2156, iou_score - 0.6631]

Epoch: 5
valid: 100%|██████████| 142/142 [00:35<00:00,  4.02it/s, dice_loss - 0.1853, iou_score - 0.7013]
best : 0.7013038690661034

Epoch: 6
valid: 100%|██████████| 142/142 [00:35<00:00,  4.01it/s, dice_loss - 0.2005, iou_score - 0.6796]

Epoch: 7
valid: 100%|██████████| 142/142 [00:35<00:00,  4.01it/s, dice_loss - 0.193, iou_score - 0.6917]

Epoch: 8
valid: 100%|██████████| 142/14

In [ ]:
#프로젝트 중 추가로 공유된 이미지에 대한 훈련 
x_train = os.path.join(train_path, 'images_추가')
y_train = os.path.join(train_path, 'labels_추가')
UnetPlusPlus_ModelTest(x_train, y_train,'se_resnext50_32x4d', 'imagenet' , 'softmax2d')

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
valid: 100%|██████████| 142/142 [00:34<00:00,  4.07it/s, dice_loss - 0.3089, iou_score - 0.5451]
best : 0.5450787257770419

Epoch: 1
valid: 100%|██████████| 142/142 [00:34<00:00,  4.09it/s, dice_loss - 0.5818, iou_score - 0.2788]

Epoch: 2
valid: 100%|██████████| 142/142 [00:35<00:00,  4.06it/s, dice_loss - 0.5924, iou_score - 0.3056]

Epoch: 3
valid: 100%|██████████| 142/142 [00:34<00:00,  4.07it/s, dice_loss - 0.4013, iou_score - 0.4772]

Epoch: 4
valid: 100%|██████████| 142/142 [00:34<00:00,  4.06it/s, dice_loss - 0.3067, iou_score - 0.5769]
best : 0.5768843245968015

Epoch: 5
valid: 100%|██████████| 142/142 [00:34<00:00,  4.08it/s, dice_loss - 0.3225, iou_score - 0.5586]

Epoch: 6
valid: 100%|██████████| 142/142 [00:34<00:00,  4.06it/s, dice_loss - 0.3211, iou_score - 0.5521]

Epoch: 7
valid: 100%|██████████| 142/142 [00:34<00:00,  4.07it/s, dice_loss - 0.3341, iou_score - 0.5376]

Epoch: 8
valid: 100%|██████████| 142/142 [00:35<00:00,  4.06it/s, dice_loss - 0.3011, iou_s

In [ ]:
#기본적으로 제공된 이미지+추가로 공유된 이미지에 대한 훈련  
x_train = os.path.join(train_path, 'images_원본+추가')
y_train = os.path.join(train_path, 'labels_원본+추가')
UnetPlusPlus_ModelTest(x_train, y_train,'se_resnext50_32x4d', 'imagenet' , 'softmax2d')

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
valid: 100%|██████████| 142/142 [00:35<00:00,  4.01it/s, dice_loss - 0.6286, iou_score - 0.2519]
best : 0.25189888587510084

Epoch: 1
valid: 100%|██████████| 142/142 [00:35<00:00,  4.01it/s, dice_loss - 0.2276, iou_score - 0.6586]
best : 0.6585736129905136

Epoch: 2
valid: 100%|██████████| 142/142 [00:35<00:00,  4.03it/s, dice_loss - 0.2685, iou_score - 0.6029]

Epoch: 3
valid: 100%|██████████| 142/142 [00:35<00:00,  4.03it/s, dice_loss - 0.1961, iou_score - 0.6914]
best : 0.6914487305871198

Epoch: 4
valid: 100%|██████████| 142/142 [00:35<00:00,  4.05it/s, dice_loss - 0.1867, iou_score - 0.7001]
best : 0.700091074260188

Epoch: 5
valid: 100%|██████████| 142/142 [00:35<00:00,  4.05it/s, dice_loss - 0.2041, iou_score - 0.6765]

Epoch: 6
valid: 100%|██████████| 142/142 [00:35<00:00,  4.06it/s, dice_loss - 0.2193, iou_score - 0.6564]

Epoch: 7
valid: 100%|██████████| 142/142 [00:35<00:00,  4.04it/s, dice_loss - 0.1884, iou_score - 0.6966]

Epoch: 8
valid: 100%|██████████| 142/14

In [ ]:
#16분할 이미지를 800개 추가한 것에 대한 훈련 
x_train = os.path.join(train_path, 'images')
y_train = os.path.join(train_path, 'labels')
UnetPlusPlus_ModelTest(x_train, y_train,'se_resnext50_32x4d', 'imagenet' , 'softmax2d')

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
valid: 100%|██████████| 142/142 [00:34<00:00,  4.08it/s, dice_loss - 0.2041, iou_score - 0.6815]
best : 0.6815179286288542

Epoch: 1
valid: 100%|██████████| 142/142 [00:34<00:00,  4.10it/s, dice_loss - 0.1703, iou_score - 0.7221]
best : 0.7220878370211155

Epoch: 2
valid: 100%|██████████| 142/142 [00:34<00:00,  4.07it/s, dice_loss - 0.1731, iou_score - 0.7151]

Epoch: 3
valid: 100%|██████████| 142/142 [00:34<00:00,  4.09it/s, dice_loss - 0.1637, iou_score - 0.7296]
best : 0.7296247272424291

Epoch: 4
valid: 100%|██████████| 142/142 [00:35<00:00,  4.05it/s, dice_loss - 0.182, iou_score - 0.7053]

Epoch: 5
valid: 100%|██████████| 142/142 [00:34<00:00,  4.06it/s, dice_loss - 0.1617, iou_score - 0.734]
best : 0.7340276182537349

Epoch: 6
valid: 100%|██████████| 142/142 [00:34<00:00,  4.07it/s, dice_loss - 0.1548, iou_score - 0.7437]
best : 0.7437055926297753

Epoch: 7
valid: 100%|██████████| 142/142 [00:34<00:00,  4.07it/s, dice_loss - 0.1625, iou_score - 0.7302]

Epoch: 8
valid:

In [ ]:
#64분할 이미지를 800개 추가된 것에 대한 훈련 
train_path = os.path.join(workspace_path, 'training_set')
x_train = os.path.join(train_path, 'images_64분할')
y_train = os.path.join(train_path, 'labels_64분할')
UnetPlusPlus_ModelTest(x_train, y_train,'se_resnext50_32x4d', 'imagenet' , 'softmax2d')

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument


Epoch: 0
valid: 100%|██████████| 142/142 [00:33<00:00,  4.22it/s, dice_loss - 0.1857, iou_score - 0.6991]
best : 0.6990774441772786

Epoch: 1
valid: 100%|██████████| 142/142 [00:33<00:00,  4.22it/s, dice_loss - 0.1575, iou_score - 0.7384]
best : 0.7384195963681582

Epoch: 2
valid: 100%|██████████| 142/142 [00:33<00:00,  4.22it/s, dice_loss - 0.1482, iou_score - 0.7503]
best : 0.7503210257896236

Epoch: 3
valid: 100%|██████████| 142/142 [00:33<00:00,  4.21it/s, dice_loss - 0.1435, iou_score - 0.7571]
best : 0.7570746442381765

Epoch: 4
valid: 100%|██████████| 142/142 [00:33<00:00,  4.21it/s, dice_loss - 0.1397, iou_score - 0.7639]
best : 0.7639108247740167

Epoch: 5
valid: 100%|██████████| 142/142 [00:33<00:00,  4.21it/s, dice_loss - 0.1453, iou_score - 0.757]

Epoch: 6
valid: 100%|██████████| 142/142 [00:33<00:00,  4.21it/s, dice_loss - 0.1447, iou_score - 0.7559]

Epoch: 7
valid: 100%|██████████| 142/142 [00:33<00:00,  4.20it/s, dice_loss - 0.131, iou_score - 0.7767]
best : 0.7767141

## 모델 생성 및 훈련

In [ ]:
ENCODER = 'efficientnet-b7'
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'softmax' 
DEVICE = 'cuda'


In [ ]:
model = smp.UnetPlusPlus(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES)+1, 
    activation=ACTIVATION,
)
preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

In [ ]:
train_dataset = Dataset(
    x_train, 
    y_train, 
    augmentation=get_training_augmentation(), 
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=CLASSES,
)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)

/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:262: FutureWarning: IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead
  warnings.warn("IAAAdditiveGaussianNoise is deprecated. Please use GaussNoise instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:385: FutureWarning: This IAAPerspective is deprecated. Please use Perspective instead
  warnings.warn("This IAAPerspective is deprecated. Please use Perspective instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1805: FutureWarning: This class has been deprecated. Please use RandomBrightnessContrast
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/albumentations/imgaug/transforms.py:232: FutureWarning: IAASharpen is deprecated. Please use Sharpen instead
  warnings.warn("IAASharpen is deprecated. Please use Sharpen instead", FutureWarning)
/usr/local/lib/python3.7/dist-packages/albument

In [ ]:
valid_dataset = Dataset(
    x_valid, 
    y_valid, 
    preprocessing=get_preprocessing(preprocessing_fn),
    classes=CLASSES,
)

valid_loader = DataLoader(valid_dataset, batch_size=1, shuffle=False, num_workers=1)

In [ ]:
loss = smp.utils.losses.DiceLoss()
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
]
optimizer = torch.optim.Adam([dict(params=model.parameters(), lr=0.0001),
])

In [ ]:
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

In [ ]:
save_dir = os.path.join(workspace_path, 'ckpt')
os.makedirs(save_dir, exist_ok=True)

In [ ]:
max_score = 0

for i in range(0, 100):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(valid_loader)

    if max_score < valid_logs['iou_score']:
        max_score = valid_logs['iou_score']
        torch.save(model, os.path.join(save_dir, 'best_model_최종2.pth'))
        print('Model saved!')

    print(max_score)

print('max_score='+str(max_score) )


Epoch: 0
train:   0%|          | 0/267 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/segmentation_models_pytorch/base/modules.py:115: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.activation(x)


valid: 100%|██████████| 142/142 [00:31<00:00,  4.46it/s, dice_loss - 0.1978, iou_score - 0.7034]
Model saved!
0.7033818583463278

Epoch: 1
valid: 100%|██████████| 142/142 [00:31<00:00,  4.46it/s, dice_loss - 0.1729, iou_score - 0.724]
Model saved!
0.7239521571867905

Epoch: 2
valid: 100%|██████████| 142/142 [00:31<00:00,  4.44it/s, dice_loss - 0.1637, iou_score - 0.7196]
0.7239521571867905

Epoch: 3
valid: 100%|██████████| 142/142 [00:31<00:00,  4.44it/s, dice_loss - 0.1359, iou_score - 0.7773]
Model saved!
0.7773037879189977

Epoch: 4
valid: 100%|██████████| 142/142 [00:31<00:00,  4.44it/s, dice_loss - 0.1308, iou_score - 0.7809]
Model saved!
0.7808535117918338

Epoch: 5
valid: 100%|██████████| 142/142 [00:31<00:00,  4.45it/s, dice_loss - 0.1232, iou_score - 0.7914]
Model saved!
0.7914499658304204

Epoch: 6
valid: 100%|██████████| 142/142 [00:32<00:00,  4.43it/s, dice_loss - 0.1222, iou_score - 0.7927]
Model saved!
0.7926560345459991

Epoch: 7
valid: 100%|██████████| 142/142 [00:31<00

## 이미지 시각화

In [ ]:
best_model = torch.load(os.path.join(save_dir, 'best_model.pth'))

In [ ]:
valid_dataset_vis = Dataset(
    x_valid, y_valid, 
    classes=CLASSES,
)

In [ ]:

for i in range(3):
    n = np.random.choice(len(valid_dataset))
    image, gt_mask = valid_dataset[n]
    gt_mask = gt_mask.squeeze()
    image_vis = valid_dataset_vis[n][0].astype('uint8')

    x_tensor = torch.from_numpy(image).to(DEVICE).unsqueeze(0)
    pr_mask = best_model.predict(x_tensor)
    pr_mask = (pr_mask.squeeze().cpu().numpy().round())

    pr_mask = np.moveaxis(pr_mask, -1, 0)
    gt_mask = np.moveaxis(gt_mask, -1, 0)
    pr_mask = np.moveaxis(pr_mask, -1, 0).squeeze()
    gt_mask = np.moveaxis(gt_mask, -1, 0).squeeze()

    pr_mask*=255
    pr_mask=pr_mask.astype(np.uint8)

    mixed = image_vis.copy()

    hsv_img = cv2.cvtColor(pr_mask, cv2.COLOR_BGR2HSV)

    mask = cv2.inRange(hsv_img, (-20, 100, 100), (20, 255, 255))
    mask_inv = cv2.bitwise_not(mask)
    roi = mixed[0:1024, 0:1024]
    fg = cv2.bitwise_and(pr_mask, pr_mask, mask=mask_inv)
    bg = cv2.bitwise_and(roi, roi, mask=mask)
    mixed[0:1024, 0:1024] = fg + bg

    visualize(
        image=image_vis,
        mixed = mixed,
        gt_mask=gt_mask,
        pr_mask=pr_mask,
        gt_building_mask=gt_mask[..., 0],
        pr_building_mask=pr_mask[..., 0],
        gt_road_mask=gt_mask[..., 1],
        pr_road_mask=pr_mask[..., 1],
    )